# Test Environment
data\
    talent.xlsx\
    waffen.xlsx\
    zauber.xlsx

einheit\
    held1.xlsx\
    held1_inventar.xlsx

In [1]:
# Modul um Würfel zu simulieren
import random
random.seed(1321345589)

# Modul zum vereinfachten Auslesen und Bearbeiten von Datentabellen
import pandas as pd

# Dies und Das
import regex as re

In [2]:
def w20():
    """
    Simple Simulation eines W20 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 20 zurück.
    """

    return random.randint(1, 20)


def w6():
    """
    Simple Simulation eines W6 Würfels.
    Benötigt keinen Input und gibt immer einen Wert zwischen 1 und 6 zurück.
    """

    return random.randint(1, 6)


def datei_zu_dataframe(path, index_column=0):
    """
    
    """

    dataframe = pd.read_excel(path, index_col=index_column)
    dataframe.dropna(inplace=True)

    return dataframe


def datei_zu_dict(path):
    """
    Liest excel Tabelle mit Informationen zum Helden ein, damit diese als
    Klassen-Attribute übergeben werden können.
    
    : name : Dateiname bzw. Name des Helden, Pfad in /einheit/name
    : return : dict
    """

    heldenframe = datei_zu_dataframe(path)
    heldendict = heldenframe.T.to_dict(orient="records")[0]

    return heldendict

In [3]:
# Einlesen der Datentabelle zu den Fertigkeiten
dsa5_talente = datei_zu_dataframe("data/dsa5_talente.xlsx", index_column=1)

# Um den späteren Zugriff zu erleichtern wird die Spalte Eigenschaftsprobe geändert:
# Die Werte sind beim Einlesen ein String in der Form 'P1/P2/P3', dieser wird am Seperator
# '/' gesplittet. Damit wird eine Liste erzeugt mit den drei Elementen [P1, P2, P3].
dsa5_talente["Eigenschaftsprobe"] = dsa5_talente.Eigenschaftsprobe.apply(lambda x: x.split("/"))

print("Die ersten 10 Zeilen der Datentabelle der Fertigkeiten")
dsa5_talente.head(10)

Die ersten 10 Zeilen der Datentabelle der Fertigkeiten


,Kategorie,Steigerungsfaktor,Eigenschaftsprobe
Talent,,,
Fliegen,Körpertalente,C,"[MU, IN, GE]"
Gaukeleien,Körpertalente,A,"[MU, CH, FF]"
Klettern,Körpertalente,B,"[MU, GE, KK]"
Körperbeherrschung,Körpertalente,B,"[GE, GE, KO]"
Kraftakt,Körpertalente,B,"[KO, KK, KK]"
Reiten,Körpertalente,B,"[CH, GE, KK]"
Schwimmen,Körpertalente,B,"[GE, KO, KK]"
Selbstbeherschung,Körpertalente,D,"[MU, MU, KO]"
Singen,Körpertalente,A,"[KL, CH, KO]"


## Glossar

In [4]:
# Kurzes Glossar durch die ganzen Abkürzungen (besser für die Textausgabe)
# Benutzt ein dictionary, also glossar["MU"] ergibt als Ausgabe "Mut"
glossar = datei_zu_dict("data/glossar.xlsx")

# Das Glossar wird verdoppelt, wobei die Dopplung Keys und Values miteinander vertauscht.
# Also das dict glossar ist nach dem "glossar[key] = value"-Prinzip aufgebaut, dies wird
# mit der dict comprehension (also {val: key for ...}) gedreht. Das daraus entstehende
# neue dict wird dann dem "alten" Glossar beigefügr (der Operator | verbindet zwi dicts.
# Damit ist jetzt auch glossar["Mut"] = "MU" gewährleistet.
glossar = glossar | {val: key for key, val in glossar.items()}

In [5]:
data_margot = datei_zu_dict("data/held_margot.xlsx")
data_margot

{'name': 'Margot',
 'alter': 31,
 'rasse': 'Mensch',
 'heldentyp': 'Schütze',
 'MU': 8,
 'KL': 8,
 'IN': 8,
 'CH': 8,
 'FF': 8,
 'GE': 8,
 'KO': 8,
 'KK': 8,
 'LeP': 21,
 'AsP': 4,
 'Int': 12,
 'AP': 0}

In [6]:
class Einheit():
    def __init__(self, filepath):
        if filepath is not None:
            file = datei_zu_dict(filepath)

            for key, value in file.items():
                setattr(self, key, value)
    
    
    def aktion(self, other, aktion):
        pass


    def reaktion(self, other):
        pass


    def eigenschaftsprobe(self, probe: str, bias: int=0):
        wurf = w20()

        if wurf > getattr(self, glossar[probe]) + bias:
            print(f"Eigenschaftsprobe misslungen! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} nicht unterworfen!")
        
        else:
            print(f"Eigenschaftsprobe geschafft! Der nötige {probe}-Wert von {getattr(self, glossar[probe])} wurde durch {wurf} unterworfen!")
    

    def fertigkeitsprobe(self, probe: str, bias: int=0):
        """
        
        """

        ausgleich = random.randint(3, 7)
        print(f'Starte Fertigkeitsprobe {probe} auf {dsa5_talente["Eigenschaftsprobe"][probe]}, Ausgleich: {ausgleich}')

        for i, eigenschaft in enumerate(dsa5_talente["Eigenschaftsprobe"][probe]):
            wurf = w20()
            print(eigenschaft)
            print(glossar[eigenschaft])
            print(f'\n{i+1}. Teilprobe auf {glossar[eigenschaft]}: Mit einem W20 wurde {wurf} geworfen!')

            ziel = getattr(self, eigenschaft) + bias
            wurfdifferenz = ziel - wurf
            print(f'Ziel: {ziel}\nDifferenz: {wurfdifferenz}')

            if wurfdifferenz < 0:
                ausgleich += wurfdifferenz
                print(f'neuer Ausgleich: {ausgleich}')
            
                if ausgleich < 0:
                    print(f'\nFertigkeitsprobe misslungen! {glossar[eigenschaft]} wurde mit {wurf} nicht unterworfen (Ziel: {getattr(self, eigenschaft)} mit Bias {bias}), Ausgleich wurde aufgebraucht ({ausgleich}).')

                    return None
                
                else:
                    print(f'Teilprobe fehlgeschlagen! {glossar[eigenschaft]} wurde mit {wurf} nicht unterworfen (Ziel: {getattr(self, eigenschaft)} mit Bias {bias}), verbleibender Ausgleich: {ausgleich}.')
            
            else:
                print(f'Teilprobe geglückt! {glossar[eigenschaft]} wurde mit {wurf} unterworfen (Ziel: {getattr(self, eigenschaft)} mit Bias {bias}), verbleibender Ausgleich: ({ausgleich}).')
        
        print(f'\nFertigkeitsprobe geglückt! Übriger Ausgleich bei {ausgleich}.')

In [7]:
margot = Einheit("data/held_margot.xlsx")

margot.fertigkeitsprobe("Tierkunde")

Starte Fertigkeitsprobe Tierkunde auf ['MU', 'MU', 'CH'], Ausgleich: 7
MU
Mut

1. Teilprobe auf Mut: Mit einem W20 wurde 19 geworfen!
Ziel: 8
Differenz: -11
neuer Ausgleich: -4

Fertigkeitsprobe misslungen! Mut wurde mit 19 nicht unterworfen (Ziel: 8 mit Bias 0), Ausgleich wurde aufgebraucht (-4).
